# ガバナンス状態を検証する
データガバナンス機能で作成したデータやGakunin RDMで付与したメタデータを検証、実験の再現性を確認するタスクです。

## 検証画面（dg-web）を表示する

In [ ]:
# 検証画面（dg-web）を表示する
import sys

import panel as pn
from IPython.display import display
from IPython.core.display import Javascript

sys.path.append('../../../../../..')
from library.utils.html.button import create_button
from library.utils.config import message as msg_config

def access_validate():
    button_width = 500
    url = 'https://web.it2.dg.nii.ac.jp/'
    obj = create_button(
        url=url,
        target='_blank',
        msg=msg_config.get('task', 'open_validate'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_validate()

## 検証結果を確認する

検証結果の一覧を表示します。<br>
再現性検証が実施されている場合、検証実施直後には検証結果が得られない可能性があります。しばらく時間を置いてから再度確認してください。<br>

In [ ]:
# 検証一覧を表示する
import os
import traceback
import json
import panel as pn
import pandas as pd

from requests.exceptions import RequestException
from IPython.core.display import Javascript
from IPython.display import display, clear_output

from library.utils import dg_web
from library.utils.error import InputWarning, UnauthorizedError, NotFoundURLError, UnusableVault
from library.task_director import TaskDirector
from library.utils.input import get_token, get_project_id
from library.utils.widgets import MessageBox
from library.utils.config import message as msg_config

notebook_name = 'validate.ipynb'

class ValidationResult(TaskDirector):
    def __init__(self, working_path: str) -> None:
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900

        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

    @TaskDirector.task_cell("2")
    def generateFormScetion(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()
        # フォーム定義
        try:
            self.define_form()
            self.done_task()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(str(e))
        except InputWarning as e:
            self._msg_output.update_warning(str(e))
            self.log.warning(str(e))
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(message)
            self.log.error(str(e))
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension("tabulator")
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def define_form(self):
        self.token = get_token()
        self.project_id = get_project_id()
        content = self.get_validate()
        self.display_table(content)
        clear_output()

    def get_validate(self):
        """検証結果一覧を取得する"""
        try:
            content = dg_web.get_validations(dg_web.SCHEME, dg_web.DOMAIN, self.token, self.project_id)
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            raise InputWarning(message)
        except NotFoundURLError:
            message = msg_config.get('form', 'project_id_not_exist').format(self.project_id)
            raise InputWarning(message)

        return content

    def display_table(self, content: dict):
        """検証結果一覧を表示する"""
        self.id_list, status_list = self.get_table(content)

        df = pd.DataFrame({
            msg_config.get('dg_web', 'validate_id'): self.id_list,
            msg_config.get('dg_web', 'status'): status_list
        })

        select_table = pn.widgets.Tabulator(df, show_index=False, header_align='center', configuration={'columnDefaults': {'headerSort': False}}, editors={'検証ID': None, '状態': None})
        select_table.style.map(self.text_color)
        select_table.on_click(self.click)

        self._form_box.append(select_table)

    @TaskDirector.callback_form('検証結果詳細を表示する')
    def click(self, event):
        """検証結果詳細を表示する"""
        try:
            validation_id = self.id_list[event.row]
            validation_result = dg_web.get_validations_validationId(dg_web.SCHEME, dg_web.DOMAIN, self.token, self.project_id, validation_id)
            validation_detail = json.dumps(validation_result, ensure_ascii=False, indent=4)

            self._msg_output.clear()
            title = '# ' + msg_config.get('dg_web', 'validation_result')
            self._msg_output.append(pn.pane.Markdown(title))
            styles = {
                'background': '#f5f5f5',
                'border': '1px solid black',
                'padding': '10px'
            }
            self._msg_output.append(pn.pane.Str(validation_detail, styles=styles))

        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)

    def text_color(self, text):
        """表のテキストの色を変更する"""
        color = ''
        if text == 'SUCCESS':
            color = 'lime'
        elif text == 'FAILURE':
            color = 'red'
        else:
            color = 'black'
        return 'color: %s' % color

    def get_table(self, content):
        """一覧に表示させる要素取得
        ID、statusの要素が無かった場合、処理をスキップする
        """
        id_list = []
        status_list = []

        for key, value in content.items():
            for table in value:
                if table.get('id') == None or table.get('status') == None:
                    continue
                else:
                    id_list.append(table.get('id'))
                    status_list.append(table.get('status').upper())
        return id_list, status_list

ValidationResult(working_path=os.path.abspath('__file__')).generateFormScetion()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector

script_file_name = "validate"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()